# Исследование надёжности заёмщиков

Первый проект в рамках обучения на курсе аналитика данных. 

В данном проекте для кредитного отдела банка было необходимо выяснить, есть ли зависимость между семейным положением и количеством у заемщика детей на факт погашения кредита в срок. От банка была предоставлена статистика о платежеспособности клиентов. 

Полученные результаты предполагается использовать для построении модели кредитного скоринга - системы, позволяющей оценить способность потенциального клиента возвращать займы банку в срок. 

## Загрузка данных и изучение общей информации

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

1) передо мной таблица с вводными от банка, в которой 12 столбцов <br>
2) в десяти столбцах все значения заполнены, в двух оставшихся (days_employed и total_income) есть пропуски (заполнено 19351 строк из 21525)<br>
3) названия столбцов не содержат никаких отклонений, с ними будет удобно работать <br>
4) на первый взгляд тип данных в столбцах совпадает с содержанием <br>
5) в столбце days_employed есть отрицательные значения, а также достаточно большие значения для трудового стажа в днях, например, в 5 строке - 340266.072047. возможно, эти значения указаны в часах 

## Предобработка данных

### Обработка пропусков

In [3]:
#смотрю уникальные значения в столбцах 
print('Уникальные значения в столбце children', data['children'].unique())
print('Уникальные значения в столбце days_employed', data['days_employed'].unique())
print('Уникальные значения в столбце dob_years', data['dob_years'].unique())
print('Уникальные значения в столбце education', data['education'].unique())
print('Уникальные значения в столбце education_id', data['education_id'].unique())
print('Уникальные значения в столбце family_status', data['family_status'].unique())
print('Уникальные значения в столбце family_status_id', data['family_status_id'].unique())
print('Уникальные значения в столбце gender', data['gender'].unique())
print('Уникальные значения в столбце income_type', data['income_type'].unique())
print('Уникальные значения в столбце debt', data['debt'].unique())
print('Уникальные значения в столбце total_income', data['total_income'].unique())
print('Уникальные значения в столбце purpose', data['purpose'].unique()) 


Уникальные значения в столбце children [ 1  0  3  2 -1  4 20  5]
Уникальные значения в столбце days_employed [-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853]
Уникальные значения в столбце dob_years [42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
Уникальные значения в столбце education ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
Уникальные значения в столбце education_id [0 1 2 3 4]
Уникальные значения в столбце family_status ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
Уникальные значения в столбце family_status_id [0 1 2 3 4]
Уникальные значения в столбце gender ['F' 'M' 'XNA']
Уникальные значени

In [4]:
#меняю отрицательное значение в графе дети на положительное
data['children'] = data['children'].abs()
#проверяю результат
data['children'].value_counts()


0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [5]:
#меняю отрицательные значения в графе трудового стажа на положительные
data['days_employed'] = data['days_employed'].abs()
#проверяю результат
data['days_employed'].value_counts()

986.927316     1
7026.359174    1
4236.274243    1
6620.396473    1
1238.560080    1
              ..
2849.351119    1
5619.328204    1
448.829898     1
1687.038672    1
582.538413     1
Name: days_employed, Length: 19351, dtype: int64

In [6]:
#смотрю количество пропущенных значений по столбцам 
data.isna().sum()
data[data['days_employed'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [7]:
#заполняю пропуски в столбце трудового стажа медианой, группируя по столбцу возраста
print('Пропуски в стаже до:', data['days_employed'].isna().sum())

for dob_year in data['dob_years'].unique():
    median = data.loc[data['dob_years'] == dob_year, 'days_employed'].median()
    #print(dob_year, median)
    data.loc[(data['days_employed'].isna()) & (data['dob_years'] == dob_year), 'days_employed'] = median

print('Пропуски после:', data['days_employed'].isna().sum())

Пропуски в стаже до: 2174
Пропуски после: 0


In [8]:
#заполняю пропуски в столбце дохода медианой, выполняя в цикле группировку по типу дохода 
print('Пропуски в доходе до:', data['total_income'].isna().sum())
for income_type in data['income_type'].unique():
    median = data.loc[data['income_type'] == income_type, 'total_income'].median()
    #print(income_type, median)
    data.loc[(data['total_income'].isna()) & (data['income_type'] == income_type), 'total_income'] = median
#смотрю результат
print('Пропуски после:', data['total_income'].isna().sum())

Пропуски в доходе до: 2174
Пропуски после: 0


In [9]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [10]:
#зсмотрю данные стажа для разных категорий дохода
data.pivot_table(index = 'income_type', values = 'days_employed').reset_index()

,income_type,days_employed
0,безработный,366413.652744
1,в декрете,3296.759962
2,госслужащий,6328.451772
3,компаньон,3861.355338
4,пенсионер,352288.489626
5,предприниматель,165883.412375
6,сотрудник,4708.358626
7,студент,578.751554


In [11]:
#функция для приведения стажа у безработных и пенсионеров в единый формат с остальными типами стажа
def hours_to_days(income_type):
    if (income_type == 'безработный') or (income_type == 'пенсионер'):
        return 24
    else: 
        return 1
data['if_hours'] = data['income_type'].apply(hours_to_days)
data['days_employed'] = data['days_employed'] / data['if_hours']
data.pivot_table(index = 'income_type', values = 'days_employed').reset_index()

,income_type,days_employed
0,безработный,15267.235531
1,в декрете,3296.759962
2,госслужащий,6328.451772
3,компаньон,3861.355338
4,пенсионер,14678.687068
5,предприниматель,165883.412375
6,сотрудник,4708.358626
7,студент,578.751554


**Вывод**

1) методом unique() выявлено, что в столбце дети есть значения -1 и 20, скорее всего, это ошибка в данных. Так как нам для результата исследования важно, есть ли у клиента дети, поменяла отрицательное значение на положительное (-1 на 1), 20 не трогала. в столбце dob, возраст клиента в годах, есть значение равное нулю, ни на что менять его не буду, оно не влияет на результат исследования. в столбце образование обнаружен разный регистр для одних и тех же значений, который считывается как разные, надо привести их к единому виду. в столбце пол встречается непонятное значение XNA, трогать его не буду. в столбце трудового стажа были обнаружегны отрицательные значения, которые методом abs() я заменила на положительные

2) методом isna() обнаружены пропуски (NaN) в столбцах days_employed и total_income. причем, их число в обоих столбцах совпадает, т.е. можно допустить, что либо по этим клиентам утеряны данные, либо у них нет трудового стажа и, соответственно, дохода. данные в этих столбцах являются количественными, относятся к вещественному типу. 

3) при выводе первых 10 строк таблицы с пропущенными значениями видно, что остальные значения в столбцах заполнены и несут в себе вполне определенную информацию. таким образом, скорее всего эти данные просто утеряны, хотя можно делать допуск на то, что люди работают неофициально и не могут подтвердить ни свой стаж, ни доход. пропуски буду заполнять медианными значениями

4) пропуски в трудовом стаже заполнила медианным значением с группировкой по столбца возраста, предположив, что трудовой стаж у людей одного возраста может быть примерно одинаковым 

5) аналогично, группируя по столбцу тип дохода, считаю медианные значения дохода и подставляю их вместо пропущенных значений. остановилась на столбце тип дохода, предположив, что для данной ситуации он может дать более-менее подходящие значения. 

### Замена типа данных

In [12]:
#меняю типы данных с вещественного на целочисленный
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
if_hours            21525 non-null int64
dtypes: int64(8), object(5)
memory usage: 2.1+ MB


**Вывод**

метод .to_numeric() переводит строковые в числовой формат float

метод .astype() переводит данные в необходимый мне тип, в качестве аргумента в методе использую int - целые числа. 
т.к. необходимо перевести вещественный float тип данных в целочисленный int, поэтому буду использовать второй метод.
результат проверяб методом info(): float64 не осталось

### Обработка дубликатов

In [13]:
#заменяю регистр в столбце образование на нижний, проверяю уникальные значения
print('уникальные значения образования ДО:', data['education'].unique())
data['education'] = data['education'].str.lower()
print('уникальные значения образования ПОСЛЕ:', data['education'].unique())


уникальные значения образования ДО: ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
уникальные значения образования ПОСЛЕ: ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


In [14]:
#ищу полные дубликаты в данных и удаляю их
print('полных дубликатов ДО:', data.duplicated().sum())
data = data.drop_duplicates()
print('полных дубликатов ПОСЛЕ:', data.duplicated().sum())
#удаляю дубликаты в данных, делая гипотезу о том, что заемщики с совпадающим возрастом, доходом, трудовым стажем и целью кредита - это дублирование данных
data = data.drop_duplicates(subset = ['dob_years', 'days_employed', 'total_income', 'purpose'])

полных дубликатов ДО: 71
полных дубликатов ПОСЛЕ: 0


In [15]:
#проверяю остались ли дубликаты по указанному условию
data[data.duplicated(subset = ['dob_years', 'days_employed', 'total_income', 'purpose'], keep = False)]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,if_hours


In [16]:
#удалила появившиеся на месте дубликатов пропущенные значения и скинула их индексацию
data = data.dropna().reset_index(drop = True)


In [17]:
#проверяю, сколько дублированных строк удалилось 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20994 entries, 0 to 20993
Data columns (total 13 columns):
children            20994 non-null int64
days_employed       20994 non-null int64
dob_years           20994 non-null int64
education           20994 non-null object
education_id        20994 non-null int64
family_status       20994 non-null object
family_status_id    20994 non-null int64
gender              20994 non-null object
income_type         20994 non-null object
debt                20994 non-null int64
total_income        20994 non-null int64
purpose             20994 non-null object
if_hours            20994 non-null int64
dtypes: int64(8), object(5)
memory usage: 2.1+ MB


**Вывод**

методы .duplicated() и .drop_duplicates() избавляют лишь от полных дубликатов. 

после приведения столбца образование к нижнему регистру методом .str.lower(), количество полных дубликатов в данных было 71 (до - 54), методом .drop_duplicates() удалила их.

в представленных данных у заемщиков нет идентификаторов, поэтому нельзя быть уверенными в том, что совпадение данных в столбцах говорит о дублировании этих данных. но предположим, что помимо полных дубликатов есть еще дублированные значения: если в данных заемщиков идет полное совпадение по возрасту, доходу, трудовому стажу и цели кредита, то это будем считать за дублирование. 

не забываю, что на месте удаленных дубликатов остаются пустые значения и занятые индексы: удаляю методом dropna() пустые значения и методом reset_index() обновляю индексацию в данных.

методом info() проверяю, сколько в данных стало строк: от изначальных 21525 осталось 20994, т.о. удалилось 531 дублированное значение.

причиной возникновения дубликатов может быть некорректное заполнение данных от заемщиков, либо сбор данных из разных источников, при соединении которых возникло дублирование.

### Лемматизация

In [18]:
#импортирую pymystem3 для лемматизации
from pymystem3 import Mystem
m = Mystem() 
from collections import Counter

In [19]:
#создаю функцию, в которой лемматизирую входящий аргумент
def purpose_category(row):
    lemmas = m.lemmatize(row)
#удаляю лишние символы в списке лемм
    for element in lemmas:
        if element == ' ':
            lemmas.remove(' ')   
    lemmas.remove('\n')
#смотрю результат лемматизации
    #print(lemmas)
    #print(Counter(lemmas))
#прописываю условие для разделения целей кредита на категории в зависимости от леммы    
    if 'свадьба' in lemmas:
        return 'свадьба'
    if 'образование' in lemmas:
        return 'образование'
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    else:
        return 'недвижимость'
    
#к исходным данным добавляю столбец с категоризацией целей кредита, применяя ранее описанную функцию
data['category_purpose'] = data['purpose'].apply(purpose_category)       
        
#смотрю результат
data.head(20)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,if_hours,category_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,1,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,1,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,24,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,1,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,1,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,1,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,1,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,1,недвижимость


**Вывод**

задание: выделить леммы в значениях столбца purpose и описать процесс лемматизации 

первым делом изучаю все уникальные значения в столбце цели, их можно разделить на 4 группы: 

1) жилье-недвижимость: покупка жилья, операции с жильем, покупка жилья для семьи, покупка недвижимости, покупка коммерческой недвижимости, покупка жилой недвижимости, строительство собственной недвижимости, недвижимость, строительство недвижимости, операции с коммерческой недвижимостью, строительство жилой недвижимости, жилье, операции со своей недвижимостью, покупка своего жилья, операции с недвижимостью, покупка жилья для сдачи, ремонт жилью 
2) автомобиль: приобретение автомобиля, на покупку подержанного автомобиля, на покупку своего автомобиля, автомобили, сделка с подержанным автомобилем, автомобиль, свой автомобиль, сделка с автомобилем, на покупку автомобиля
3) образование: дополнительное образование, образование, заняться образованием, получение образования, получение дополнительного образования, получение высшего образования, профильное образование, высшее образование, заняться высшим образованием
4) свадьба: сыграть свадьбу, на проведение свадьбы, свадьба.

для лемматизации описала функцию, которая в качестве аргумента принимает строки из столбца purpose, далее, в зависимости от леммы, функция будет возвращать одно из четырех интересующих нас значений. дополнительно избавилась в леммах от пробелов и незначимых символов

применив методом apply() описанную функцию к столбцу цель кредита, добавила в исходные данные новый стоблец с категорией цели, результат проверила методом info()

### Категоризация данных

In [20]:
#смотрю соответстиве семейного положения его айдишнику
family = data[['family_status', 'family_status_id']]
family = family.drop_duplicates().reset_index()
family.head()

,index,family_status,family_status_id
0,0,женат / замужем,0
1,4,гражданский брак,1
2,18,вдовец / вдова,2
3,19,в разводе,3
4,24,Не женат / не замужем,4


In [21]:
print(data['debt'].value_counts())

0    19300
1     1694
Name: debt, dtype: int64


In [22]:
#категоризирую наличие детей
def have_children(children):
    if children < 1:
        return 'no'
    else:
        return 'yes'

data['having_children'] = data['children'].apply(have_children)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,if_hours,category_purpose,having_children
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,недвижимость,yes
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,1,автомобиль,yes
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1,недвижимость,no
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,1,образование,yes
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,24,свадьба,no


In [23]:
#категоризирую уровень дохода

#ввожу переменные, возвращающие значения персентилей для столбца дохода, используя метод describe()
a = data['total_income'].describe()['25%']
b = data['total_income'].describe()['50%']
c = data['total_income'].describe()['75%']
  

In [24]:
#описываю функцию, с помощью которой можно провести категоризацию дохода
def income_id(total_income):
    if total_income <= a:
        return 'менее 25%'
    if a < total_income <= b:
        return 'от 25 до 50%'
    if b < total_income <= c:
        return 'от 50 до 75%'
    else:
        return 'свыше 75%'

#применяю функцию к столбцу доход и смотрю результат
data['category_income'] = data['total_income'].apply(income_id)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,if_hours,category_purpose,having_children,category_income
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,недвижимость,yes,свыше 75%
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,1,автомобиль,yes,от 25 до 50%
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1,недвижимость,no,от 50 до 75%
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,1,образование,yes,свыше 75%
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,24,свадьба,no,от 50 до 75%


**Вывод**

для ответа на последующие вопросы данные надо категоризировать. 

1) наличие детей. необходимо добавить столбец having_children, yes - если есть дети, no - если нет

2) семейное положение. семейное положение уже категоризировано, для понимания сопоставления семейного положения идентификатору создала новую таблицу family, которую сгруппировала, чтоб были видны индексы для family_status. upd: мне этот столбец не пригодился, сводную таблицу выводила по столбцу family_status

3) уровень дохода: можно разделить на категории персентилями: меньше 25%, от 25 до 50, от 50 до 75 и от 75 до 100%

4) цели кредитования уже разделены на категории 

все категории будем сравнивать со столбцом debt, в котором два значения: 0 - не было задолженности, 1 - была. 

## Ответы на поставленные вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [25]:
#создаю сводную таблицу по наличию детей и долга
children_pivot = data.pivot_table(index = 'having_children', columns = 'debt', values = 'dob_years', aggfunc = 'count')
children_pivot.columns
#добавляю колонку с процентом долга для дальнейшего анализа
children_pivot['%_debt'] = children_pivot[1] / children_pivot[0] * 100
children_pivot.head()

debt,0,1,%_debt
having_children,,,
no,12785,1038,8.118889
yes,6515,656,10.069071


**Вывод**

сделала сводную таблицу, которая отображает наличие детей и подсчитывает, были ли задолженности. 
из этой таблицы видно, что примерно у 2/3 заемщиков детей нет, из тех, у кого детей нет задолженность по кредиту возникала у примерно 8%, а у тех, у кого дети есть - около 10%. можно сказать, что бездетные платят кредит вовремя в большем числе случаев, нежели те, у кого есть дети

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [26]:
#создаю сводную таблицу по семейному положению и долгу
fam_status_pivot = data.pivot_table(index = 'family_status', columns = 'debt', values = 'dob_years', aggfunc = 'count')
#добавляю колонку с процентом долга для дальнейшего анализа
fam_status_pivot['%_debt'] = fam_status_pivot[1] / fam_status_pivot[0] * 100
fam_status_pivot.head()

debt,0,1,%_debt
family_status,,,
Не женат / не замужем,2478,271,10.936239
в разводе,1087,85,7.819687
вдовец / вдова,881,62,7.037457
гражданский брак,3673,371,10.100735
женат / замужем,11181,905,8.094088


**Вывод**

в данном случае вывожу сводную таблицу, отображающую наличие или отсутствие задолженности у категорий граждан с разным семейным положением. 
чаще всего не выплачивают кредит люди из категории не женат/не замужем и находящиеся в гражданском браке, лучше всего платят вдовцы

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [27]:
#создаю сводную таблицу по уровню дохода и долгу
income_pivot = data.pivot_table(index = 'category_income', columns = 'debt', values = 'dob_years', aggfunc = 'count')
#добавляю колонку с процентом долга для дальнейшего анализа
income_pivot['%_debt'] = income_pivot[1] / income_pivot[0] *100
income_pivot.head()

debt,0,1,%_debt
category_income,,,
менее 25%,4838,411,8.495246
от 25 до 50%,4788,460,9.607352
от 50 до 75%,4800,448,9.333333
свыше 75%,4874,375,7.693886


**Вывод**

сводная таблица по категориям уровня дохода и наличию долга показывает, что чаще всего платят вопремя люди с высоким доходом (выше 75% от среднего), в данной группе показатели хуже всего у тех, чей доход распределен в категорию 25-50%

### Как разные цели кредита влияют на его возврат в срок?

In [28]:
purp_pivot = data.pivot_table(index = 'category_purpose', columns = 'debt', values = 'dob_years', aggfunc = 'count')
purp_pivot['%_debt'] = purp_pivot[1] / purp_pivot[0] * 100
purp_pivot.head()

debt,0,1,%_debt
category_purpose,,,
автомобиль,3837,390,10.164191
недвижимость,9808,765,7.799755
образование,3566,361,10.123388
свадьба,2089,178,8.520823


**Вывод**

меньше всего просрочек по выплатам у людей, чьей целью были операции с недвижимостью, а хуже отдают за авто и образование.

## Общий вывод

Если для банка являются значительными процентные значения в погашении кредита вплоть до 2-3 единиц, то сводные таблицы помогут сделать вывод о том, что лучше всего отдают кредиты: <br>
1) заемщики без детей (8% долгов против 10% у имеющих детей)<br>
2) разведенные и вдовы/вдовцы (7-7,8% долгов против 10,9% у неженатых/незамужних)<br>
3) люди с высоким доходом (7,6% против 9.3-9.6% обладжателей среднего уровня дохода)<br>
4) заемщики, берущие кредит на операции с недвижимостью (7,8% против 10.1% у заемщиков с целью кредитования автомодиль и образование)<br>
Могу отметить, что при анализе данных не обнаружено слишком больших выбросов для категорий заемщиков, возврат кредита в срок характерен для 90-93% случаев.

## Чек-лист готовности проекта

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.